In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Load the trained model
model = load_model('mnist_noisy_16x16_classifier.h5')

# Function to load and preprocess an image
def load_and_preprocess_image(file_path):
    img = Image.open(file_path).convert('L')  # Convert to grayscale
    img_array = np.array(img) / 255.0  # Normalize to [0, 1]
    return img_array.reshape(1, 16, 16, 1)  # Reshape for model input

# Function to predict digit
def predict_digit(image):
    prediction = model.predict(image)
    return np.argmax(prediction), np.max(prediction)

# Path to your test data folder
test_data_folder = 'test_data'

# Lists to store results
true_labels = []
predicted_labels = []
confidences = []
file_names = []

# Process each image in the test_data folder
for filename in os.listdir(test_data_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Add other extensions if needed
        file_path = os.path.join(test_data_folder, filename)
        
        # Load and preprocess the image
        img = load_and_preprocess_image(file_path)
        
        # Predict
        pred_label, confidence = predict_digit(img)
        
        # Extract true label from filename (assuming format: 'digit_*.png')
        true_label = int(filename.split('_')[0])
        
        # Store results
        true_labels.append(true_label)
        predicted_labels.append(pred_label)
        confidences.append(confidence)
        file_names.append(filename)

# Convert to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate accuracy
accuracy = np.mean(true_labels == predicted_labels)
print(f"Overall Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, range(10))
plt.yticks(tick_marks, range(10))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

# Add text annotations to the confusion matrix
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

# Print detailed results
print("\nDetailed Results:")
for true, pred, conf, fname in zip(true_labels, predicted_labels, confidences, file_names):
    print(f"File: {fname}, True: {true}, Predicted: {pred}, Confidence: {conf:.2f}")

# Plot some misclassified images
misclassified = np.where(true_labels != predicted_labels)[0]
num_to_plot = min(10, len(misclassified))

plt.figure(figsize=(15, 2 * num_to_plot))
for i, idx in enumerate(misclassified[:num_to_plot]):
    img = load_and_preprocess_image(os.path.join(test_data_folder, file_names[idx]))
    plt.subplot(num_to_plot, 5, i*5 + 1)
    plt.imshow(img.reshape(16, 16), cmap='gray')
    plt.title(f"True: {true_labels[idx]}, Pred: {predicted_labels[idx]}")
    plt.axis('off')

plt.tight_layout()
plt.show()